<a href="https://colab.research.google.com/github/swishswish123/tracked_surgery_simulations/blob/main/tracked_endoscope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2) TRACKED ENDOSCOPE SIMULATION

In this notebook we want to study the accuracy of a tracked endoscope, and what the expected accuracy of an overlay will be. The overlay accuracy could be measured in terms of accuracy in 3D in the endoscope camera frame, or in terms of 2D pixel accuracy.

Please first read tracked_pointer.ipynb before this notebook for a better understanding of the problem and the maths behind it.

## imports and utility functions


In [ ]:
# github repo which contains all the images
!git clone https://github.com/swishswish123/tracked_surgery_simulations.git repo


## Background, System Layout, goal and assumptions

The following diagram shows the layout of the navigation system and the different components involved in the surgery when using a tracked endoscope for AR.



In [ ]:
Image(filename="./repo/assets/endoscope_setup.png")


The goal is to be able to display some segmented piece of information from the pre-operative MRI in MRI coordinates onto the endoscopic video:

In [ ]:
Image(filename="./repo/assets/endoscope_goal.png")


The goal is therefore turning some segmented MRI coordinates to Endoscopic Image (EndIm) coordinates.

X<sub>EndIm</sub> = T X<sub>MRI</sub>

where T is composed of the following transforms:

T = <sup>EndIm</sup>T<sub>EndP</sub> * 
    <sup>EndP</sup>T<sub>EndRef</sub> * 
    <sup>EndRef</sup>T<sub>Cam</sub>  * 
    <sup>Cam</sup>T<sub>PatRef</sub> * 
    <sup>PatRef</sup>T<sub>MRI</sub>
    


Here is an visual representation of the endoscope setup with all the transforms:


In [ ]:
Image(filename="endosim_demo/images/endoscope_setup_transforms.png")

In the following sections we will go step by step on how to go from each of these transforms

## Assumptions

In order to make this a like-for-like comparison to the tracked pointer simulation, we should keep most of the reference data identical. The difference now, is we swap a pointer for an endoscope. We will need an additional hand-eye calibration, and reference data to project from camera space onto image space, but we leave all other data the same. 

In [ ]:
Image(filename="endosim_demo/images/endoscope_setup_assumptions.png")


In [ ]:
# P - , the length of the endoscope.
LENGTH_OF_ENDOSCOPE = 180 # use 300 after merging

# D - z distance from camera to plane where everything is located
DISTANCE_FROM_CAM = 2000 # since the camera and patient reference are aligned in the x and y directions, only distance is in z

# 0 - angle of pointer
ENDOSCOPE_ANGLE = 45

# Yc - distances from tumour to patient reference
TUMOUR_PATREF = 300  

# NDI quotes 0.25mm for Polaris Spectra, some papers estimate it at 0.17mm
#TYPICAL_TRACKING_SIGMA = 0.25

# For Model 2 and 3, using an endoscope, this determines the distance of a target of interest from the endoscope.
#working_distance = 50

# for simulation to be reproducible
NUMBER_SAMPLES = 10

X_T = 100 # head length (about 20cm)
Y_T = 130 # menton to top of head (about 25cm)
Z_T = 80 # head bredth (about 15cm)

END_SIGMA=0.5
SIGMA_STEP=0.01